In [ ]:
#uncomment to install packages
#conda install -c conda-forge selenium
#!pip install fake_useragent
#!pip install eventlet
#!pip install python-settings

In [1]:
from bs4 import BeautifulSoup
import requests
import time, os
import re
import random
from random import randint
import random
from time import sleep

import pandas as pd
from pymongo import MongoClient
from pprint import pprint
import json

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [3]:
from fake_useragent import UserAgent

In [4]:
#helper files
import get_proxies as proxies
import extractors

### Create Mongo collection for urls

In [5]:
client = MongoClient()

In [6]:
db = client['philadelphia_home_prices']

In [7]:
db.list_collection_names()

['prices_messy', 'listing_urls', 'listing_data']

In [8]:
#uncomment to delete the collection
#db.drop_collection('listing_urls')

In [9]:
db.list_collection_names()

['prices_messy', 'listing_urls', 'listing_data']

In [10]:
col = db.create_collection('listing_urls')

CollectionInvalid: collection listing_urls already exists

In [11]:
db.list_collection_names()

['prices_messy', 'listing_urls', 'listing_data']

### scrape pages for listing urls

In [ ]:
url = 'https://www.realtor.com/'

In [ ]:
# 44 listings per page. Choose start and stop page
page_from = 26
page_to = 26

In [ ]:
col = db.get_collection('listing_urls')

In [ ]:
col.count()

In [ ]:
for i in range(page_from, page_to+1):
    #print(i) #prints page that is open in case bot is detected
    driver = webdriver.Chrome(chromedriver)
    driver.get(url + 'soldhomeprices/Philadelphia_PA/pg-' + str(i))
    
    sleep(random.uniform(60,120))
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    page = str(soup.find_all('div', class_='photo-wrap'))
    extensions = re.findall(r'[A-z]+-[A-z]+\/[0-9-A-z]+[^"]', page)
    
    if len(extensions) == 0:
        print('bot is caught on page {}'.format(i))
        break
        
    for i in extensions:
        page_url = url + i
        page = {'url': page_url}
        col.insert_one(page)
        
    sleep(random.uniform(1,3))
    
    driver.close()

In [ ]:
col.count()

### Pull urls from mongoDB into memory

In [12]:
#pull urls from mongo db
cursor = db.listing_urls.find({})
listings = list(cursor)

In [13]:
#convert dictionary values to list
listing_urls_list = []
for i in listings:
    url = i['url']
    listing_urls_list.append(url)

In [14]:
#see first 5 in list
listing_urls_list[0:5]

['https://www.realtor.com/realestateandhomes-detail/1232-N-Mascher-St-Apt-A_Philadelphia_PA_19122_M49182-95881',
 'https://www.realtor.com/realestateandhomes-detail/212-Carpenter-St-Rear-C_Philadelphia_PA_19147_M34206-29412',
 'https://www.realtor.com/realestateandhomes-detail/1211-E-Cheltenham-Ave_Philadelphia_PA_19124_M33700-20336',
 'https://www.realtor.com/realestateandhomes-detail/1002-Brandywine-St_Philadelphia_PA_19123_M39147-93479',
 'https://www.realtor.com/realestateandhomes-detail/2607-N-31st-St_Philadelphia_PA_19132_M32883-74202']

### Get data from urls

#### new collection for listing data

In [15]:
#uncomment the code you need to use
#db.create_collection('listing_data')
col = db.get_collection('listing_data')

In [16]:
db.collection_names()

/Users/chrismalasics/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  """Entry point for launching an IPython kernel.


['prices_messy', 'listing_urls', 'listing_data']

In [17]:
#uncomment to drop
#db.drop_collection('listing_data')

In [18]:
db.collection_names()

/Users/chrismalasics/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  """Entry point for launching an IPython kernel.


['prices_messy', 'listing_urls', 'listing_data']

#### Rotate proxies

In [19]:
#get list of proxies from free-proxy.net
proxies_address = proxies.proxy_list()

2020-11-04 16:19:02,103 root   DEBUG    === Initialized Proxy Parsers ===
2020-11-04 16:19:02,104 root   DEBUG    	 FreeProxy parser of 'http://free-proxy-list.net' with required bandwidth: '150' KBs
2020-11-04 16:19:02,104 root   DEBUG    	 PremProxy parser of 'https://premproxy.com/list/' with required bandwidth: '150' KBs
2020-11-04 16:19:02,105 root   DEBUG    =================================
2020-11-04 16:19:02,610 root   DEBUG    Added 300 proxies from FreeProxy
2020-11-04 16:19:03,368 http_request_randomizer.requests.parsers.PremProxyParser DEBUG    Pages: {'', '05.htm', '03.htm', '02.htm', '11.htm', '13.htm', '08.htm', '09.htm', '07.htm', '06.htm', '12.htm', '10.htm', '04.htm'}
2020-11-04 16:19:03,954 http_request_randomizer.requests.parsers.js.UnPacker INFO     JS UnPacker init path: https://premproxy.com/js/c5d9a.js
2020-11-04 16:19:04,559 http_request_randomizer.requests.parsers.js.UnPacker DEBUG    portmap: {'r6a39': '8080', 'rb7d0': '32108', 'reb9c': '51671', 'r6838': '47

In [31]:
listing_from = 13
listing_to = len(listing_urls_list)
count = 0
tested_proxies = []
bad_proxies = []
bad_url = []

In [21]:
import sys
import concurrent.futures
from datetime import datetime
import eventlet
import settings

In [ ]:
pool = eventlet.GreenPool(settings.max_threads)
pile = eventlet.GreenPile(pool)

In [ ]:
datetime.now()

In [ ]:
proxies

In [ ]:
for i in range(listing_from, listing_to+1): 
    print(i, listing_urls_list[i])

    try:
        #PROXY = proxies[count].get_address()
        PROXY = proxies_address[count]
        webdriver.DesiredCapabilities.CHROME['proxy']={
                                                        "httpProxy":PROXY,
                                                       "ftpProxy":PROXY,
                                                       "sslProxy":PROXY,
                                                       "proxyType":"MANUAL",
                                                        }
        driver = webdriver.Chrome(chromedriver)
        driver.get(listing_urls_list[i])

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        listing_data = get_data(soup)
        good_proxies.append(PROXY)

    except:
        bad_proxies.append(PROXY)
        count += 1
        #driver.close()
        
    driver.close()
    
    #insert listing data into mongo database
    col.insert_one(listing_data)
    print(listing_data)
    count += 1
    print(count)

In [ ]:
threads = min(MAX_THREADS, len(story_urls))

with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
    executor.map(download_url, story_urls)

In [22]:
from extractors import get_data 

In [ ]:
count

In [32]:
for i in range(listing_from, listing_to+1): 
    print(i, listing_urls_list[i])
    check = False
    state = False
    while state==False:
        try:
            #PROXY = proxies[count].get_address()
            PROXY = proxies_address[count]
            print(PROXY)
            webdriver.DesiredCapabilities.CHROME['proxy']={
                                                            "httpProxy":PROXY,
                                                           "ftpProxy":PROXY,
                                                           "sslProxy":PROXY,
                                                           "proxyType":"MANUAL",
                                                            }
            driver = webdriver.Chrome(chromedriver)
            driver.get(listing_urls_list[i])

            soup = BeautifulSoup(driver.page_source, 'html.parser')
            listing_data = get_data(soup)
            
            #try:
            #    check = (soup.find('span', itemprop="streetAddress")).text
            #except:
            #    bad_url.append(listing_urls_list[i])
            #    break
      
                
            
            tested_proxies.append(PROXY)
            state = True
        except:
            bad_proxies.append(PROXY)
            count += 1
            driver.close()
        
    driver.close()
    
    #insert listing data into mongo database
    col.insert_one(listing_data)
    print(listing_data)
    count += 1
    print(count)
    
    
    
    

    
    #if i//5 == 0:
    #    sleep(random.uniform(60,120))
        
    ####     
    #session=requests.Session()
    #session.headers = user_agent#{'User-Agent': ua.random} #'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    #print(user_agent)
    ##base_url='https://www.realtor.com'
    ##re_list=[]
    ##realtor_default_count=44
    ##page_counter=1
    ##search_url=''.join([base_url, '/realestateandhomes-search/Sacramento_CA'])
    #page=session.get(listing_urls_list[i])
    #c=page.content
    #soup=BeautifulSoup(c,"html.parser")
    ####
    
    #options = Options()
    #ua = UserAgent()
    #userAgent = ua.random
    #print(userAgent)
    #options.add_argument(f'user-agent={userAgent}')
    #driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)
    #driver.get(listing_urls_list[i])
    
    #sleep(random.uniform(120,180)) #pause

13 https://www.realtor.com/realestateandhomes-detail/606-E-Olney-Ave_Philadelphia_PA_19120_M44163-32382
77.77.215.6:8080


2020-11-04 16:25:46,001 selenium.webdriver.remote.remote_connection DEBUG    POST http://127.0.0.1:61016/session {"capabilities": {"firstMatch": [{}], "alwaysMatch": {"browserName": "chrome", "platformName": "any", "proxy": {"httpProxy": "77.77.215.6:8080", "ftpProxy": "77.77.215.6:8080", "sslProxy": "77.77.215.6:8080", "proxyType": "manual"}, "goog:chromeOptions": {"extensions": [], "args": []}}}, "desiredCapabilities": {"browserName": "chrome", "version": "", "platform": "ANY", "proxy": {"httpProxy": "77.77.215.6:8080", "ftpProxy": "77.77.215.6:8080", "sslProxy": "77.77.215.6:8080", "proxyType": "MANUAL"}, "goog:chromeOptions": {"extensions": [], "args": []}}}
2020-11-04 16:25:46,910 selenium.webdriver.remote.remote_connection DEBUG    Finished Request
2020-11-04 16:25:46,911 selenium.webdriver.remote.remote_connection DEBUG    POST http://127.0.0.1:61016/session/a073f8b24ee6e94a2bc7f8dcf22336fe/url {"url": "https://www.realtor.com/realestateandhomes-detail/606-E-Olney-Ave_Philadelph

124.6.129.135:80


2020-11-04 16:26:18,157 selenium.webdriver.remote.remote_connection DEBUG    POST http://127.0.0.1:61047/session {"capabilities": {"firstMatch": [{}], "alwaysMatch": {"browserName": "chrome", "platformName": "any", "proxy": {"httpProxy": "124.6.129.135:80", "ftpProxy": "124.6.129.135:80", "sslProxy": "124.6.129.135:80", "proxyType": "manual"}, "goog:chromeOptions": {"extensions": [], "args": []}}}, "desiredCapabilities": {"browserName": "chrome", "version": "", "platform": "ANY", "proxy": {"httpProxy": "124.6.129.135:80", "ftpProxy": "124.6.129.135:80", "sslProxy": "124.6.129.135:80", "proxyType": "MANUAL"}, "goog:chromeOptions": {"extensions": [], "args": []}}}
2020-11-04 16:26:19,059 selenium.webdriver.remote.remote_connection DEBUG    Finished Request
2020-11-04 16:26:19,060 selenium.webdriver.remote.remote_connection DEBUG    POST http://127.0.0.1:61047/session/74f5aaff116e4366af2970d13d90237c/url {"url": "https://www.realtor.com/realestateandhomes-detail/606-E-Olney-Ave_Philadelph

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=61047): Max retries exceeded with url: /session/74f5aaff116e4366af2970d13d90237c/window (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fd7ca959590>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [33]:
tested_proxies

[]

In [26]:
get_data(soup)

{'Address': '606 E Olney Ave,',
 'Locality': 'Philadelphia',
 'Region': 'PA',
 'Postal Code': '19120',
 'Price': ['175,000',
  '175,000',
  '175,000',
  '175,000',
  '175,000',
  '175,000',
  '175,000',
  '175,000'],
 'Days on Market': ['43'],
 'SQFT House': ['1,088', '1,088', '1,088', '1,088'],
 'SQFT Lot': ['1760', '1760'],
 'Full Baths': ['2'],
 'Half Baths': [],
 'Bedrooms': ['3'],
 'Garage': [],
 'Master Bath': [],
 'Cooling': [' Window Unit(s)'],
 'Heating': [' Forced Air Heating'],
 'Annual Tax': ['1422'],
 'House Type': [' Interior Row/Townhouse'],
 'Build Year': [' 1930', ' 1930'],
 'Updated': [' 08/07/2020'],
 'Neighborhood': ['OLNEY'],
 'Neighborhood median sales': ['123,000'],
 'Neighborhood Median DOM': ['67'],
 'Neighborhood Price SQFT': ['113'],
 'Association': ['No'],
 'Association Monthly': ['0'],
 'crawl time': datetime.datetime(2020, 11, 4, 16, 21, 50, 858056)}

In [27]:
tested_proxy

NameError: name 'tested_proxy' is not defined

In [ ]:
hello = {'hi':[]}

In [ ]:
hello['hi'] == []

In [ ]:
import re

In [ ]:
get_data(soup)

In [ ]:
bad_proxies

In [ ]:
for i in range(listing_from, listing_to+1): 
    
    print(i, listing_urls_list[i])
    
    #ua = UserAgent()
    #user_agent = {'User-agent': ua.random}
    #print(user_agent)

    #response  = requests.get(listing_urls_list[i], headers = user_agent)
    
    #driver = webdriver.Chrome(chromedriver)
    #driver.get(listing_urls_list[i])
    
    #if i//5 == 0:
    #    sleep(random.uniform(60,120))
        
    ####     
    #session=requests.Session()
    #session.headers = user_agent#{'User-Agent': ua.random} #'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    #print(user_agent)
    ##base_url='https://www.realtor.com'
    ##re_list=[]
    ##realtor_default_count=44
    ##page_counter=1
    ##search_url=''.join([base_url, '/realestateandhomes-search/Sacramento_CA'])
    #page=session.get(listing_urls_list[i])
    #c=page.content
    #soup=BeautifulSoup(c,"html.parser")
    ####
    
    #options = Options()
    #ua = UserAgent()
    #userAgent = ua.random
    #print(userAgent)
    #options.add_argument(f'user-agent={userAgent}')
    #driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)
    #driver.get(listing_urls_list[i])

    ###
    
    '''options = Options()
    ua = UserAgent()
    userAgent = ua.random
    #print(userAgent)
    options.add_argument(f'user-agent={userAgent}')
    driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)
    driver.get(listing_urls_list[i])
    #user_agent = {'User-agent': 'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)'}
    #response  = requests.get(listing_urls_list[i], headers = user_agent)
    
    sleep(random.uniform(60,100))'''
    ###
    
   
    
    PROXY = proxies[count].get_address()
    
    
    print(PROXY)
    webdriver.DesiredCapabilities.CHROME['proxy']={
                                                    "httpProxy":PROXY,
                                                    "ftpProxy":PROXY,
                                                    "sslProxy":PROXY,

                                                    "proxyType":"MANUAL",

                                                    }
    driver = webdriver.Chrome(chromedriver)
    driver.get(listing_urls_list[i])
    
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    

    driver.close()
    
    #insert listing data into mongo database
    col.insert_one(listing_data)
    print(listing_data)
    count += 1
    
    #sleep(random.uniform(120,180)) #pause

In [ ]:
proxies

In [ ]:
listing_data

In [ ]:
test = ('hi',50)

In [ ]:
test[1]

In [ ]:
type(test)

In [ ]:
def test(num):
    return num+12, num-12

In [ ]:
plus, minus = test(12)

In [ ]:
plus

In [ ]:
minus

In [ ]:
proxies[1].get_address()

In [ ]:
proxies

In [ ]:
PROXY = proxies[0].get_address()

In [ ]:
PROXY

In [ ]:
webdriver.DesiredCapabilities.CHROME['proxy']={
                                                            "httpProxy":PROXY,
                                                           "ftpProxy":PROXY,
                                                           "sslProxy":PROXY,
                                                           "proxyType":"MANUAL",
                                                            }

In [ ]:
driver = webdriver.Chrome(chromedriver)
driver.get(listing_urls_list[0])

In [ ]:
listing_urls_list[0]

In [ ]:
type(listing_urls_list)

In [ ]:
listing_urls_list[:20]

In [ ]:
soup

In [ ]:
try:
    check = (soup.find('span', itemprop="streetAddress")).text
except:

In [ ]:
proxies_address

In [ ]:
street_address = ('Not Found')
test = {'street_address': street_address}

In [ ]:
listing_data['street_address'] == 'Not Found'

In [ ]:
type(soup)